# Drift MPC

In [4]:
from mg_utils import *

In [ ]:
# casadi/acados model
import casadi as ca
from casadi import SX, sqrt, atan, tan, sin, cos, tanh, atan2, fabs, vertcat, if_else, sign
from acados_template import AcadosModel, AcadosOcp, AcadosOcpSolver, AcadosSim, AcadosSimSolver
# suppress warnings acados
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='acados_template')


def STM_model():
    # variables
    v = SX.sym('v') # velocity
    v̇ = SX.sym('v̇') # velocity dot
    β = SX.sym('β') # sideslip angle
    β̇ = SX.sym('β̇') # sideslip angle dot
    r = SX.sym('r') # yaw rate
    ṙ = SX.sym('ṙ') # yaw rate dot
    δ = SX.sym('δ') # wheel angle (on the road)
    Fx = SX.sym('Fx') # rear longitudinal force

    x = vertcat(v, β, r) # state vector
    ẋ = vertcat(v̇, β̇, ṙ) # state dot vector
    u = vertcat(Fx, δ) # u input vector 

    # tire model
    αf = δ - atan2(v*sin(β) + a*r, v*cos(β))
    αr = -atan2(v*sin(β) - b*r, v*cos(β))
    def fiala_tanh_ca(α, Fx, Fz, μ, Cy):
        Fy_max = sqrt(μ**2 * Fz**2 - Fx**2) # maximum lateral force
        αs = atan(Fy_max/Cy) # maximum slip angle
        return Fy_max * tanh(α / αs) # tanh approximation
    def fiala_ca(α, Fx, Fz, μ, Cy):
        Fy_max = sqrt(μ**2 * Fz**2 - Fx**2) # maximum lateral force
        Fy_lin = Cy * tan(α) - Cy**2 * fabs(tan(α)) * tan(α) / (3 * Fy_max) + Cy**3 * tan(α)**3 / (27 * Fy_max**2)
        Fy_sat = Fy_max * sign(α)
        return if_else(fabs(α) < atan(3*Fy_max/Cy), Fy_lin, Fy_sat)

    # choose the tire model
    # def tire(α, Fx, Fz, μ, Cy): return fiala_tanh_ca(α, Fx, Fz, μ, Cy) # choose the tire model
    def tire(α, Fx, Fz, μ, Cy): return fiala_ca(α, Fx, Fz, μ, Cy) # choose the tire model

    # lateral forces
    Fyf = tire(αf, 0.0, Fz_Front, μf, Cyf) # lateral force front
    Fyr = tire(αr, Fx, Fz_Rear, μr, Cyr) # lateral force rear
    Fxr = Fx # rear longitudinal force

    # define the symbolic equations of motion
    dv = (-Fyf*sin(δ-β) + Fxr*cos(β) + Fyr*sin(β)) / m # V dot
    dβ = (+Fyf*cos(δ-β) - Fxr*sin(β) + Fyr*cos(β)) / (m*v) - r # β dot
    dδ = (a*Fyf*cos(δ) - b*Fyr) / J_CoG # r dot

    dx = vertcat(dv, dβ, dδ) # equations of motion

    # create the model
    aca_model = AcadosModel()
    aca_model.name = 'stm_model'
    aca_model.f_impl_expr = ẋ - dx  # implicit dynamics
    aca_model.f_expl_expr = dx  # explicit dynamics
    aca_model.x = x  # state vector
    aca_model.xdot = ẋ  # state dot vector
    aca_model.u = u  # u input vector
    return aca_model

# create the model
aca_model = STM_model()

In [6]:
# utils
def clear_previous_simulation():
    variables = globals().copy()
    for v in variables:
        if(type(variables[v]) is AcadosSimSolver or type(variables[v]) is AcadosOcpSolver):
            del globals()[v]

def piecewise_constant(setpoints, setpoints_duration, Ts):
    # compute the number of samples for each setpoint 
    n_samples = np.rint(setpoints_duration/Ts).astype(int)
    # compute the reference for each setpoint
    ref = setpoints[0]*np.ones((n_samples[0],))
    for i in range(1, len(setpoints)):
        ref = np.append(ref, setpoints[i]*np.ones((n_samples[i],)))
    # add one sample to account for the non-strict inequality on both sides in the definition of the last setpoint 
    ref = np.append(ref, ref[-1])
    # compute final time instant
    Tf = np.round(np.sum(n_samples)*Ts, 3)
    return (ref, Tf)

def compute_num_steps(ts_sim, Ts, Tf):
    assert np.isclose(Ts % ts_sim, 0), "The sample time Ts has to be an integer multiple of the simulation time step ts_sim"
    assert np.isclose(Tf % ts_sim, 0), "The simulation time Tf has to be an integer multiple of the simulation time step ts_sim"
    N_steps    = int(Tf/ts_sim) # compute the number of simulation steps
    N_steps_dt = int(Tf/Ts) # compute the number of steps for the discrete-time part of the loop
    n_update   = int(Ts/ts_sim) # number of simulation steps every which to update the discrete-time part of the loop 
    return (N_steps, N_steps_dt, n_update)

In [ ]:
clear_previous_simulation()
# define simulation fundamental time step [s]
ts_sim = 0.001

# setup controller parameters
# - system model
model = STM_model()
# - controller sample time [s]
Ts = 0.01
# - number of shooting time intervals 
N  = 50
# - prediction horizon length [s]
T = N*Ts

T_tot = 10 # total simulation time [s]

# model used to simulate the system
sim_model = STM_model()

# Equilibrium point (found in matlab)
V_eq = 4.86 # [m/s] total velocity
beta_eq = -0.44 # [rad] sideslip angle
r_eq = 1.39 # [rad/s] yaw rate
delta_eq = -0.13 # [rad] steering angle 
Fx_eq = 46.29 # [N] longitudinal force

# Input bounds
delta_lb = -MAX_DELTA # lower bound on steering angle
delta_ub = MAX_DELTA  # upper bound on steering angle
Fx_lb = MIN_FX # lower bound on longitudinal force
Fx_ub = MAX_FX  # upper bound on longitudinal force

# State bounds
V_lb = MIN_V # [m/s] lower bound on velocity
V_ub = MAX_V # [m/s] upper bound on velocity

# initial condition
x0 = np.array([V_eq, beta_eq, r_eq])

# initial guess for the control inputs
u0 = np.array([delta_eq, Fx_eq]) # delta, Fx

# define cost weigth matrices
w_V     = 1
w_beta  = 1e1
w_r     = 1e1

w_delta = 1
w_Fx    = 1


Q = np.diag([w_V, w_beta, w_r])
R = np.diag([w_delta, w_Fx])


# get state and control dimensions
nx, nu = model.x.rows(), model.u.rows()

# define reference 
# - define reference for the angle and, accordingly, the simulation time Tf
# angle_ref, Tf = piecewise_constant(np.array([np.pi, 0]), np.array([5, 10]), Ts)

V_ref, Tf    = piecewise_constant(np.array([V_eq]), np.array([T_tot]), Ts)
beta_ref, _  = piecewise_constant(np.array([beta_eq]), np.array([T_tot]), Ts)
r_ref, _     = piecewise_constant(np.array([r_eq]), np.array([T_tot]), Ts)
Fx_ref, _    = piecewise_constant(np.array([Fx_eq]), np.array([T_tot]), Ts)
delta_ref, _ = piecewise_constant(np.array([delta_eq]), np.array([T_tot]), Ts)


# - provide a reference for all variables 
# y_ref = np.column_stack((np.zeros((len(angle_ref), 1)), angle_ref.reshape(-1,1), np.zeros((len(angle_ref), nx+nu-2))))
y_ref_nolookahead = np.column_stack((V_ref, beta_ref, r_ref, delta_ref, Fx_ref))
# y_ref_nolookahead = np.column_stack((vx_ref, vy_ref, r_ref, np.zeros((len(vx_ref), nu))))

# - add N samples at the end (replicas of the last sample) for reference look-ahead
y_ref = np.vstack((y_ref_nolookahead, np.repeat(y_ref_nolookahead[-1].reshape(1,-1), N, axis=0)))

# compute the number of steps for simulation
N_steps, N_steps_dt, n_update = compute_num_steps(ts_sim, Ts, Tf)

# configure whether to apply shifting and to enable reference look-ahead
shifting    = False
ref_preview = False

AttributeError: 'AcadosModel' object has no attribute 'copy'